# AutoSleep Data 


In [126]:
import pandas as pd
import numpy as np 


# First we need to import our data from a csv file: 
df = pd.read_csv("AutoSleep.csv", header=0)


## Explore the Data & Handle Missing Values
* With this dataset, it appears that the nights I wasn't wearing my watch were not recorded so there are no missing values
* There are, however, `np.NaN` values in the columns tracking blood oxygen levels because my Apple Watch doesn't have that capability. 
    * Since this data is missing throughout the entire time interval of the dataset, I will be deleting those rows from the dataframe.
* This is nice except when I want to compare it so another source of data, it will be a little more difficult to combine 
without filling in the missing dates on the autosleep data or removing those dates from the other dataset. 


In [127]:
# What are the column names?
print(df.columns)
# It might be useful if I make a table that translates some of the column nates such as "ISO8601" and "qualityAvg7"

# print(df.head(5))
# print(df.tail(5))

# this for loop returns true if a column has any null values
for col in df:
    check_for_nan = df[col].isnull().values.any()
    if check_for_nan == True:
        print (col, check_for_nan, df[col].isnull().count())
        df.drop(col, axis=1, inplace=True)

print("after dropping empty columns:", df.head())

Index(['ISO8601', 'fromDate', 'toDate', 'bedtime', 'waketime', 'inBed',
       'awake', 'fellAsleepIn', 'sessions', 'asleep', 'asleepAvg7',
       'efficiency', 'efficiencyAvg7', 'quality', 'qualityAvg7', 'deep',
       'deepAvg7', 'sleepBPM', 'sleepBPMAvg7', 'dayBPM', 'dayBPMAvg7',
       'wakingBPM', 'wakingBPMAvg7', 'hrv', 'hrvAvg7', 'SpO2Avg', 'SpO2Min',
       'SpO2Max', 'respAvg', 'respMin', 'respMax', 'tags', 'notes'],
      dtype='object')
deep True 48
deepAvg7 True 48
dayBPM True 48
dayBPMAvg7 True 48
SpO2Avg True 48
SpO2Min True 48
SpO2Max True 48
respAvg True 48
respMin True 48
respMax True 48
tags True 48
notes True 48
after dropping empty columns:                      ISO8601                fromDate                  toDate  \
0  2022-01-28T20:59:59-08:00  Thursday, Jan 27, 2022    Friday, Jan 28, 2022   
1  2022-01-29T20:59:59-08:00    Friday, Jan 28, 2022  Saturday, Jan 29, 2022   
2  2022-01-30T20:59:59-08:00  Saturday, Jan 29, 2022    Sunday, Jan 30, 2022   
3  2022-02-

The following Columns have missing values for every instance in the dataset:
* deep 
* deepAvg7 
* dayBPM 
* dayBPMAvg7 
* SpO2Avg 
* SpO2Min 
* SpO2Max 
* respAvg 
* respMin 
* respMax 
* tags 
* notes 

## Now for some Summary Statistics: 
* Summarize the following into a pandas Series:
    * `nights_total` = total number nights logged
    * `avg_sleep` = average sleep 
    * `avg_[day]` = average sleep for each day of the week
    * `std_sleep` = standard deviation of sleep 
    * `std_[day]` = standard deviation for each day of the week
    * `mode_hours` = most common sleep length (rounded to the nearest hour)
    * `mode_day` = most common day of the week when mode sleep length occurred


In [128]:
# collecting summary statistics:
import utils 
import importlib
importlib.reload(utils)

summary_stats = []

# 1. total number of nights logged
nights_total = df.shape[0]
summary_stats.append(nights_total)

# 2. average hours slept
# we need to convert the timestamps into numeric values before we can calculate the mean
clean_df = utils.clean_sleep(df, "asleep")
sleep_ser = clean_df["asleep"]   
avg_sleep_secs = sleep_ser.mean()
avg_sleep = utils.sec_to_hours(avg_sleep_secs)
median_sleep_secs = sleep_ser.median()
median_sleep = utils.sec_to_hours(median_sleep_secs)
print("average time slept:", avg_sleep, "median:", median_sleep)
summary_stats.append(avg_sleep)

# 3. average sleep for each day of the week
days_of_week = utils.separate_days(df)
week_df = pd.DataFrame(days_of_week)
print(week_df)
avg_monday = utils.sec_to_hours(week_df["Monday"].dropna().mean())
avg_tuesday =  utils.sec_to_hours(week_df["Tuesday"].dropna().mean())
print("TUESDAY", avg_tuesday)
avg_wednesday =  utils.sec_to_hours(week_df["Wednesday"].dropna().mean())
avg_thursday =  utils.sec_to_hours(week_df["Thursday"].dropna().mean())
avg_friday =  utils.sec_to_hours(week_df["Friday"].dropna().mean())
avg_saturday =  utils.sec_to_hours(week_df["Saturday"].dropna().mean())
avg_sunday =  utils.sec_to_hours(week_df["Sunday"].dropna().mean())
summary_stats.append([avg_monday, avg_tuesday, avg_wednesday, avg_thursday, avg_friday, avg_saturday, avg_sunday])

# 4. standard deviation of sleep
std_sleep = utils.sec_to_hours(sleep_ser.std())
summary_stats.append(std_sleep)
print("STD:", std_sleep)

#5. standard deviation for each day of the week
std_mon = utils.sec_to_hours(week_df["Monday"].dropna().std())
std_tues = utils.sec_to_hours(week_df["Tuesday"].dropna().std())
std_wed = utils.sec_to_hours(week_df["Wednesday"].dropna().std())
std_thurs = utils.sec_to_hours(week_df["Thursday"].dropna().std())
std_fri = utils.sec_to_hours(week_df["Friday"].dropna().std())
std_sat = utils.sec_to_hours(week_df["Saturday"].dropna().std())
std_sun = utils.sec_to_hours(week_df["Sunday"].dropna().std())
summary_stats.append([std_mon, std_tues, std_wed, std_thurs, std_fri, std_sat, std_sun])


# 6. most common sleep length rounded to the nearest hour
ser = sleep_ser.copy()
for i in range(len(sleep_ser)):
    hours = round(sleep_ser[i]/3600, 0)
    ser[i] = hours
mode_hours = ser.mode()
print("mode", mode_hours) # 8.78 is about 8 hours and 45 minutes! 
summary_stats.append(mode_hours)

# 7. most common sleep length rounded to the nearest hour for each day 
mode_mon = utils.mode_mon(week_df).mode()
mode_tues = utils.mode_tues(week_df).mode()
mode_wed = utils.mode_wed(week_df).mode()
mode_thurs = utils.mode_wed(week_df).mode()
mode_fri = utils.mode_fri(week_df).mode()
mode_sat = utils.mode_sat(week_df).mode()
mode_sun = utils.mode_sun(week_df).mode()
print("mon", mode_mon, "tues", mode_tues, "wed", mode_wed, "thurs", mode_thurs, "fri", mode_fri, "sat", mode_sat, "sun", mode_sun)
print("Monday and Sunday nights are when I get the most common amount of hours of sleep: 9.0")


average time slept: ['7.0 hours 23.0 mins 18.75 seconds'] median: ['7.0 hours 46.0 mins 0.0 seconds']
    Monday  Tuesday  Wednesday  Thursday   Friday  Saturday   Sunday
0  25200.0  34020.0    22320.0   29460.0  39480.0     24300  26760.0
1  31680.0  23400.0    21600.0   16740.0  23940.0     30540  31980.0
2  35160.0  32400.0     9000.0   13980.0  28020.0     27900  18540.0
3  26340.0  24240.0    33540.0   12000.0  29580.0      4920  31620.0
4  32520.0  23640.0    24420.0   38460.0  35760.0     13680  28080.0
5  18120.0  21180.0        NaN   23820.0      NaN     40260  29940.0
6      NaN  25260.0        NaN   30780.0      NaN     29160  32460.0
7      NaN      NaN        NaN   27840.0      NaN     30180  31620.0
8      NaN      NaN        NaN       NaN      NaN     30900      NaN
TUESDAY ['7.0 hours 18.0 mins 25.714285714286234 seconds']
STD: ['2.0 hours 7.0 mins 25.636114526029814 seconds']
mode 0    9.0
dtype: object
mon 0    7.0
1    9.0
dtype: float64 tues 0    7.0
dtype: float64 

# Apple Watch Data 

In [129]:
df2 = pd.read_csv("allWorkouts.csv", header=0)

df2.head()
df2.tail()


,Type,Start,End,Duration,Distance,Average Heart Rate,Max Heart Rate,Average Pace,Average Speed,Average Cadence,Active Energy,Total Energy,Elevation Ascended,Elevation Descended,Weather Temperature,Weather Humidity
87,Cardio Dance,2022-01-31 06:50,2022-01-31 07:00,0:09:31,NaN,147.057471,179,NaN,NaN,NaN,72.306,86.085,NaN,NaN,5.680000e-14,72.0
88,Outdoor Walking,2022-01-30 13:07,2022-01-30 13:34,0:27:20,2.102270,118.352761,137,0:13:00,4.613647,212.319542,109.177,168.186,17.77,NaN,1.000000e+00,73.0
89,High Intensity Interval Training,2022-01-30 11:23,2022-01-30 11:55,0:31:42,NaN,155.742105,183,NaN,NaN,NaN,269.338,320.952,NaN,NaN,5.680000e-14,74.0
90,Cardio Dance,2022-01-29 23:30,2022-01-29 23:47,0:16:32,NaN,147.376404,175,NaN,NaN,NaN,116.924,152.372,NaN,NaN,-3.000000e+00,87.0
91,Outdoor Walking,2022-01-29 12:24,2022-01-29 12:59,0:34:22,2.615872,114.576642,141,0:13:08,4.566826,204.922822,134.777,202.739,20.52,NaN,-2.000000e+00,79.0


## Explore the Data and Handle Missing Values

* The biggest source of missing values in this dataset comes from the `Elevation Ascended` and `Elevation Descended` columns. Since this factor is not something that I am planning on using alongside my sleep data, the best way to handle these missing values is to simply drop the columns from the dataframe. 
* The remaining missing values occur due to the different types of workouts that were recorded. For instance, yoga does not require an average distance, pace, speed, or cadence to be recorded. I think the best way to handle these instances while preserving the parts of the data that I would like to analyze is by leaving them `np.NAN` values and dealing with them on a case by case basis. 


In [130]:
# before dropping two columns, this is how many missing values there are
nan_check = df2.isnull()
for col in nan_check:
    for i in range(len(col)):
        if nan_check[col].iloc[i] == True:
            print("column:", col, "row:", i, df2["Type"].iloc[i])

df2_dropped = df2.drop(["Elevation Ascended", "Elevation Descended"], axis=1)

# after dropping the elevation columns:
print("****AFTER DROPPING****")
nan_check2 = df2_dropped.isnull()
for col in nan_check2:
    for i in range(len(col)):
        if nan_check2[col].iloc[i] == True:
            print("column:", col, "row:", i, df2_dropped["Type"].iloc[i])

df2_dropped.head()


column: Distance row: 1 Yoga
column: Average Pace row: 1 Yoga
column: Average Pace row: 10 Indoor Cycling
column: Average Speed row: 1 Yoga
column: Average Speed row: 10 Indoor Cycling
column: Average Cadence row: 1 Yoga
column: Average Cadence row: 10 Indoor Cycling
column: Average Cadence row: 13 Functional Strength Training
column: Average Cadence row: 14 Indoor Cycling
column: Elevation Ascended row: 0 Indoor Walking
column: Elevation Ascended row: 1 Yoga
column: Elevation Ascended row: 3 Outdoor Walking
column: Elevation Ascended row: 8 Outdoor Walking
column: Elevation Ascended row: 10 Indoor Cycling
column: Elevation Ascended row: 13 Functional Strength Training
column: Elevation Ascended row: 14 Indoor Cycling
column: Elevation Ascended row: 16 Yoga
column: Elevation Ascended row: 17 Yoga
column: Elevation Descended row: 0 Indoor Walking
column: Elevation Descended row: 1 Yoga
column: Elevation Descended row: 2 Outdoor Running
column: Elevation Descended row: 3 Outdoor Walking


,Type,Start,End,Duration,Distance,Average Heart Rate,Max Heart Rate,Average Pace,Average Speed,Average Cadence,Active Energy,Total Energy,Weather Temperature,Weather Humidity
0,Indoor Walking,2022-04-16 14:06,2022-04-16 14:42,0:36:29,2.357790,126.598039,149,0:15:28,3.876709,181.938358,177.101,263.491,8.0,36.0
1,Yoga,2022-04-11 06:48,2022-04-11 07:12,0:24:47,NaN,108.421818,165,NaN,NaN,NaN,105.901,145.958,-1.0,80.0
2,Outdoor Running,2022-04-10 11:13,2022-04-10 11:29,0:15:47,2.682093,184.482014,201,0:05:53,10.190971,277.027090,157.182,196.443,3.0,64.0
3,Outdoor Walking,2022-04-05 11:59,2022-04-05 12:20,0:20:49,1.345861,89.687500,114,0:15:28,3.876185,197.929515,50.862,97.588,7.0,43.0
4,Outdoor Walking,2022-04-03 18:50,2022-04-03 19:22,0:31:57,2.502640,120.893229,137,0:12:46,4.697795,202.639625,120.817,183.847,11.0,42.0


## Determining What Data To Analyze

* I want to find ways to correlate these two datasets, which meands that I need some questions that the dataset might have answers to...

* Does the amount of sleep I get affect the length of my workout? 
* Does it affect the type of workout I do?
* Does the time I do my workout at affect the time I go to sleep?
* Does the time I wake up affect when I workout? 
* Does the amount of sleep I get affect the total energy expenditure? 
    * Is there are correlation between any of these questions and a particular day of the week? (This extra layer would require a lot more data to show anything significant)

### Coming up with a hypothesis to test: 

* I want to find out if getting more sleep will lead to a more productive workout (higher total energy expenditure) 

